# Finding Similar Items
The goal of this notebook is to implement the stages of finding textually similar documents based on Jaccard similarity. Specifically, the implemented techniques to tackle the problem in a more optimal way are: shingling, minhashing, and locality-sensitive hashing (LSH).

The techniques are applied to documents collected from [ArXiv.org](https://arxiv.org/), with the goal of detecting plagiarism from a random set of real documents. In the case of papers, the similarity does not need to be very high to be considered significant. Some percentage lying around 20% would be unusual enough.

## Techniques
- **Shingling**: To be able to calculate similarity between sets, we need to first convert documents to set of elements. The most effective way would be to construct the set of short strings that appear in the document. Therefore, documents that share pieces as short as sentences or even phrases will have many common elements despite the order. To do so, we can use the shingling technique to extract the unique k-shingles that compose the document, these are the substrings of length k found within the document. Check *generate_shingles* method.
- **MinHashing**: A drawback from the set of shingles is that they can be very large to store, approximately 4 times the space the document takes. A solution is to generate signatures which can be compared between documents to estimate the Jaccard similarity of the underlying sets. Check *min_hashing* method.
- **LSH - Local Sensitive Hashing**: Even with minhashing, comparing the similarity between all documents it is very expensive computationally. LSH aims to generate, from the set of signatures, a reduced list of candidate pairs, which would be the ones which similarity must be evaluated. For signature matrices, this means to hash documents to several buckets and consider elements of the same bucket as candidate pairs. Check *lsh* method.

## Implementation
The code is implemented using Python and PySpark. In addition, Numpy is applied on all methods to try to make them as efficient as possible.

## Authors
- Serghei Socolovschi [serghei@kth.se](mailto:serghei@kth.se)
- Angel Igareta [alih2@kth.se](mailto:alih2@kth.se) 

## General

### Libraries

In [1]:
# Run only once
!pip install arxiv
!pip install pdfminer.six

     |████████████████████████████████| 81kB 3.5MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-cp36-none-any.whl size=6066 sha256=78e83683073cbb93499a5c472862d24d6b32d1382936bb88e2d46eb574230165
  Stored in directory: /root/.cache/pip/wheels/f1/80/5a/444ba08a550cdd241bd9baf8bae44be750efe370adb944506a
Successfully built sgmllib3k
     |████████████████████████████████| 5.6MB 5.5MB/s 
     |████████████████████████████████| 3.2MB 42.6MB/s 


In [2]:
import arxiv
from pdfminer.high_level import extract_text
from os import listdir
import re # Subsitute in string
import collections # Count collisions
import random
import time
import math
import numpy as np
import sys
from itertools import combinations 

### Common

## Methods

### Fetch documents
Fetch **number_of_documents** documents within the topic **topic** and stores them to the directory **dir**, previously reseting the folder.

In [3]:
def fetch_documents(number_of_documents, topic, dirpath):
  # Create folder corpus where all documents will be saved 
  !rm -rf $dirpath
  !mkdir $dirpath

  # Query papers containing quantum and save to corupus folder
  result = arxiv.query(query=topic, max_results=number_of_documents, iterative = True)
  
  # Download pdfs and store in dirpath, returning the full path
  document_paths = [arxiv.arxiv.download(paper, dirpath) for paper in result()]
  return document_paths

### Extract text from documents
Convert pdfs to text using the library pdfminer.

In [4]:
def extract_texts_from_documents(dirpath):
  texts = [extract_text(dirpath + document_path) for document_path in listdir(dirpath)]
  return texts

In [5]:
def extract_text_from_document(document_path):
  return extract_text(document_path)

### Preprocess text
It converts text to be in lowercase, with only alphanumeric characters and single spaces. This way we optimize it to have a more accurate similarity.

In [6]:
def preprocess_text(raw_text):
  # Convert the text to lowercase (to have a more accurate similarity)
  text = raw_text.lower()
  # Replace all none alphanumeric characters with spaces
  text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
  # Replace multiple spaces with one space
  text = re.sub(r'\s+', ' ', text)
  
  return text

### Generate shingles
Constructs k–shingles of a given length **k** from a given document, computes a hash value for each unique shingle, and represents the document in the form of an ordered set of its hashed k-shingles.

In [7]:
def generate_shingles(text, k):
  shingles_raw = [text[i:i + k] for i in range(len(text) - k + 1)]
  shingles = np.array(shingles_raw)

  # Remove duplicates. Do not use set or we will be loosing the order which is very important.
  _, unique_indexes = np.unique(shingles, return_index=True)
  return shingles[np.sort(unique_indexes)]

In [8]:
# Try shingles method. Expected result for string "abcdabd" k = 2 => {ab, bc, cd, da, bd}
generate_shingles("abcdabd", 2)

array(['ab', 'bc', 'cd', 'da', 'bd'], dtype='<U2')

Compress shingles of length k to integers, using a max of 4B per shingle (python integer internal size). [More Info](https://cp-algorithms.com/string/string-hashing.html)

In [9]:
def compress_shingles(shingles):
  hashed_shingles_raw = [hash(shingle) for shingle in shingles]
  hashed_shingles = np.array(hashed_shingles_raw)

  # Calculate number of collissions (before making them unique)
  # collisions = [item for item, count in collections.Counter(shingles_hashes).items() if count > 1]
  # print("Number of collisions in " + str(len(shingles_hashes)) + " hashes: " + str(len(collisions)))

  # Make them unique (in case there was any collition)
  _, unique_indexes = np.unique(hashed_shingles, return_index=True)
  return hashed_shingles[np.sort(unique_indexes.astype(int))]

In [10]:
# Check python hash does not change in same execution (same seed)
print(hash("Hello"))
print(hash("Helloo")) 

1872632765187643746
1411792157458509121


In [11]:
# Try shingles method. Expected result for string "abcdabd" k = 2 => {ab, bc, cd, da, bd}
compress_shingles(generate_shingles("abcdabd", 2))

array([-1583230123171554083, -3331476452345085769,  3572837399102241351,
        8954000560296602273, -1257682914828647777])

In [12]:
# Try shingles method. Expected result for string "abcdabd" k = 2 => {ab, bc, cd, da, bd}
[hash(shingle) for shingle in generate_shingles("abcdabd", 2)]

[-1583230123171554083,
 -3331476452345085769,
 3572837399102241351,
 8954000560296602273,
 -1257682914828647777]

### Compare two sets of shingles using Jaccard similarity

In [13]:
def get_jaccard_similarity_from_shingles(shingles_1, shingles_2):
  # Here we can use set because we are not taking into account order for the intersection
  intersection_set = np.intersect1d(shingles_1, shingles_2)

  intersection = len(intersection_set)
  union = len(shingles_1) + len(shingles_2) - intersection
  return intersection / union

### Random coefficients generator

Create lists of random integers of the length of the signature which will be furtherly used to obtain random hashing functions

In [14]:
max_shingle_hash = sys.maxsize # defines the range for random coefficients (could be changed)
def random_coefficients(signature_length):
  random_list = np.array([], dtype=np.uint8)

  for i in range(signature_length):
    random_index = random.randint(1, max_shingle_hash)
    while random_index in random_list: # Ensure uniqueness
        random_index = random.randint(1, max_shingle_hash)
    random_list = np.append(random_list, random_index)

  return random_list

### Generate minhash signature from shingles

For each hash function, all the shingles are hashed and then the min one is selected

In [15]:
def minhashing(signature_length, shingles, a_hash_coefficients, b_hash_coefficients, max_hash): 
  signature = np.array([], dtype=np.uint8)

  for i in range(0, signature_length):
    hashes = ((a_hash_coefficients[i] * shingles) + b_hash_coefficients[i]) % max_hash
    signature = np.append(signature, np.amin(hashes))

  return signature

### Similarity between minhash signatures

Calculate proportion of similar elements (minhashes) from two signatures.

In [16]:
def get_similarity_minhash_signatures(signature_1, signature_2):
  count = 0
  for i in range(0, len(signature_1)):
    if str(signature_1[i]) == str(signature_2[i]):
      count += 1

  return count / len(signature_1)

### LSH

Finding best tradeoff between the bands and the rows in order to obtain the similarity closest to the threshold.

In [17]:
def calculate_lsh_params(signature_length, threshold):
  min = 1
  result = (1, 1)

  for i in range(1, signature_length + 1):
    band_number = i
    rows_number = signature_length / i

    local_min = abs(threshold - (1 / band_number) ** (1 / rows_number))
    if min > local_min:
      min = local_min
      result = (band_number, rows_number)

  return result

Main lsh method

In [18]:
# Result => Pairs of candidates
def lsh(signatures, signature_length, threshold, lsh_buckets):
  candidates_pairs = set()
  best_params = calculate_lsh_params(signature_length, threshold)

  lsh_bands = best_params[0]
  rows_number = int(best_params[1])

  # Calculate coefficients of hash function
  a_hash_coefficients = random_coefficients(lsh_bands)
  b_hash_coefficients = random_coefficients(lsh_bands)  

  for band_index in range(lsh_bands):
    band_row_start = band_index * rows_number
    band_signature_hashes = {} # []

    # Calculate subsignature reduction or intermediate hash for each subsignature
    subsignatures_reductions_raw = [hash(str(s[band_row_start:band_row_start + rows_number])) for s in signatures]
    subsignatures_reductions = np.array(subsignatures_reductions_raw)
    subsignature_hashes = (a_hash_coefficients[band_index] * subsignatures_reductions + b_hash_coefficients[band_index]) % lsh_buckets

    # For each subsignature in the band, calculate its hash
    for subsignature_hash_index in range(len(subsignature_hashes)):
      subsignature_hash = subsignature_hashes[subsignature_hash_index]
      if subsignature_hash not in band_signature_hashes:
        band_signature_hashes[subsignature_hash] = np.array([], dtype=np.uint8)
      band_signature_hashes[subsignature_hash] = np.append(band_signature_hashes[subsignature_hash], [subsignature_hash_index])

    # Calculate signatures index with same hash (instead of saving it in memory) (optimization ignored)
    # for i in range(len(band_signature_hashes) - 1):
    #   for j in range(i + 1, len(band_signature_hashes)):
    #     if band_signature_hashes[i] == band_signature_hashes[j]:
    #       candidates_pairs.add((i, j))

    for (key, signature_indexes) in band_signature_hashes.items():
      if len(signature_indexes) >= 2:
        for candidates_pair in list(combinations(signature_indexes, 2)):
          candidates_pairs.add(candidates_pair)

  return candidates_pairs

## Wrapper methods

Get compressed shingles from raw document

In [19]:
def get_shingles_from_document(document_path, shingle_size):
  raw_text = extract_text_from_document(document_path)
  text = preprocess_text(raw_text)
  shingles = generate_shingles(text, shingle_size)
  shingles_hashes = compress_shingles(shingles)
  return shingles_hashes

## Experiments

In [20]:
corpus_folder = "./corpus/"
corpus_topic = "quantum"
corpus_size = 2
shingles_size = 10

documents_paths = fetch_documents(corpus_size, corpus_topic, corpus_folder)
document_path = documents_paths[0]
document_path

'./corpus/0201082v1.Quantum_Computers_and_Quantum_Computer_Languages_Quantum_Assembly_Language_and_Quantum_C_Language.pdf'

In [21]:
# Get text from document
raw_text = extract_text_from_document(document_path)
raw_text[:100]

'QQuuaannttuumm  CCoommppuutteerrss  aanndd  QQuuaannttuumm  CCoommppuutteerr  LLaanngguuaaggeess::\n\n'

In [22]:
# Preprocess text 
text = preprocess_text(raw_text)
text[:100]

'qquuaannttuumm ccoommppuutteerrss aanndd qquuaannttuumm ccoommppuutteerr llaanngguuaaggeess qquuaann'

In [23]:
# Get shingles from text
shingles = generate_shingles(text, shingles_size)
shingles[80:100]

array(['sssseemmbb', 'ssseemmbbl', 'sseemmbbll', 'seemmbblly',
       'eemmbbllyy', 'emmbbllyy ', 'mmbbllyy l', 'mbbllyy ll',
       'bbllyy lla', 'bllyy llaa', 'llyy llaan', 'lyy llaann',
       'yy llaanng', 'y llaanngg', 'guuaaggee ', 'uuaaggee a',
       'uaaggee aa', 'aaggee aan', 'aggee aann', 'ggee aannd'],
      dtype='<U10')

In [24]:
# Get compressed shingles
shingles_hashes = compress_shingles(shingles)
shingles_hashes[80:100]

array([ 6826790457743011032, -2981279910775453740, -7457232420149427715,
         784872998395550615, -5171530530838379130,  8388583554838212350,
        5457718796081926824,  7428560074082474786, -4941900259768505278,
        1489192709548571884, -2268647060724996796,  4626870868283178449,
        7126011376062341505,  6755775087789693078,   919741840833001063,
       -8413034045029173490,  -615193119471560707,  -899363863111132833,
        7300330558747071105,    92843432654188988])

## Main method

Configuration

In [25]:
corpus_folder = "./corpus/"
corpus_topic = "quantum"
corpus_size = 20
shingle_size = 10

# Minhash
max_hash_minhash = 4294967311 # big prime number that can be used for hashing
signature_length = 1000

# LSH Config
lsh_custom_threshold = 0.3 # It should be multiple of signature length
lsh_buckets = max_hash_minhash

In [26]:
# Print params => (band, rows_number)
calculate_lsh_params(signature_length, lsh_custom_threshold)

(223, 4.484304932735426)

In [27]:
# Get <corpus_size> documents and save them to ./corupus folder
documents_paths = fetch_documents(corpus_size, corpus_topic, corpus_folder)
documents_paths

['./corpus/0201082v1.Quantum_Computers_and_Quantum_Computer_Languages_Quantum_Assembly_Language_and_Quantum_C_Language.pdf',
 './corpus/0407102v1.Quantum_Networks_for_Generating_Arbitrary_Quantum_States.pdf',
 './corpus/0804.3401v1.Quantum_Computational_Complexity.pdf',
 './corpus/1311.4939v1.Geometrical_perspective_on_quantum_states_and_quantum_computation.pdf',
 './corpus/1611.03472v1.Universal_Quantum_Algorithm.pdf',
 './corpus/9610034v1.Quantum_Grothendieck_polynomials.pdf',
 './corpus/0302169v1.Nonlinear_Dynamics_In_Quantum_Physics_Quantum_Chaos_and_Quantum_Instantons.pdf',
 './corpus/0309066v1.Probabilistic_foundations_of_quantum_mechanics_and_quantum_information.pdf',
 './corpus/0504224v1.From_quantum_graphs_to_quantum_random_walks.pdf',
 './corpus/2006.03757v1.Quantum_Metamaterials_Applications_in_quantum_information_science.pdf',
 './corpus/1610.02500v1.Probabilistic_Process_Algebra_to_Unifying_Quantum_and_Classical_Computing_in_Closed_Systems.pdf',
 './corpus/1402.1141v1.Quan

### Calculating similarity between two documents

In [28]:
# Choose documents to run similarity
document_path_1 = documents_paths[3]
document_path_2 = documents_paths[4]
print(document_path_1)
print(document_path_2)

./corpus/1311.4939v1.Geometrical_perspective_on_quantum_states_and_quantum_computation.pdf
./corpus/1611.03472v1.Universal_Quantum_Algorithm.pdf


In [29]:
# Get set of hashed shingles of first two documents
shingles_1 = get_shingles_from_document(document_path_1, shingle_size)
shingles_2 = get_shingles_from_document(document_path_2, shingle_size)
print(shingles_1[:100])
print(shingles_2[:100])

[-4878247784018520252  6979104729595748972  8596380596457510034
  4743876708560457059  7803064971898023827 -7461869544768665117
 -5953950010689897734   458034627060557673  8973741114008401069
  6420084574154802230 -1540180902921627005  3883979735442944472
 -6403872996980931840 -7747004395348051766 -1833684814269139758
 -2693785178226882827 -5860971311104872506  6886666010101769611
  7670045090892855041  5170133178740610422 -6720917274267561127
 -1907867832306311621  2289618202077666317  8126507106362543347
 -5061219317607589564  5946374217117094395  2702652095180749099
 -1547039076773101576  5535789323999678000 -2038886991908967324
 -8370221625605229690  5325940770629868029  6074590038053326623
 -7366934327414400348  4893758671935900693  3873604022619044195
  1115725543352504636  7883359286879850303  6561944636168891177
  -809906874317581945 -3878967849330111135 -1018608119484310376
 -3842627522389050076   658819876814758027  1548556883360355929
 -8474247657565355964  36519210291861102

In [30]:
# Get jaccard similarity from hashed shingles
get_jaccard_similarity_from_shingles(shingles_1, shingles_2)

0.02079693400214169

In [31]:
# For every document to have same hash functions
a_minhash_coefficients = random_coefficients(signature_length) # lists of randomly generated coefficients that will be used for minhashing
b_minhash_coefficients = random_coefficients(signature_length)

In [32]:
# Get minhashe signatures of hashed shingles
signature_1 = minhashing(signature_length, shingles_1, a_minhash_coefficients, b_minhash_coefficients, max_hash_minhash)
signature_2 = minhashing(signature_length, shingles_2, b_minhash_coefficients, b_minhash_coefficients, max_hash_minhash)
print(signature_2[:20])

[ 516014  275677  182616 1255481  153400  233519  895793  676951 1266181
  896179  117575  725239  869575  849330  145416  703946  392495  128961
  131766   25229]


In [33]:
# Get similarity of signatures
get_similarity_minhash_signatures(signature_1, signature_2)

0.0

In [34]:
# LSH
lsh([signature_1, signature_2], signature_length, 0.03, lsh_buckets)

set()

In [35]:
# LSH
lsh([signature_1, signature_2], signature_length, 0.05, lsh_buckets)

set()

### Calculate similarity whole corpus

In [56]:
corpus_shingles = []
start = time.time()
for document_path_index in range(len(documents_paths)): 
  document_path = documents_paths[document_path_index]
  shingles = get_shingles_from_document(document_path, shingle_size)

  # Important: Increment similarity between signatures by adding the previous shingles
  # Just for forcing to be similar (it can be commented for real similarity, but in real papers it would probably not output anything)
  if document_path_index != 0:
    shingles = np.append(shingles, corpus_shingles[document_path_index - 1])
    
  corpus_shingles.append(shingles)
end = time.time()

In [57]:
print(f"Runtime of the generating shingles was {end - start}")
print(f"Average per document: {(end - start) / len(documents_paths)}")

Runtime of the generating shingles was 67.28062891960144
Average per document: 3.364031445980072


In [58]:
## Filter shingles with less than million rows
corpus_shingles = list(filter(lambda corpus_shingle: len(corpus_shingle) > 100000, corpus_shingles))

## Trim shingles to max 200000, so they are between 100k and 200k
corpus_shingles = [corpus_shingle[:150000] for corpus_shingle in corpus_shingles]

In [59]:
# For every document to have same hash functions
a_minhash_coefficients = random_coefficients(signature_length) # lists of randomly generated coefficients that will be used for minhashing
b_minhash_coefficients = random_coefficients(signature_length)

In [60]:
corpus_signatures = []
start = time.time()

for shingle in corpus_shingles:
  signature = minhashing(signature_length, shingle, a_minhash_coefficients, b_minhash_coefficients, max_hash_minhash)
  corpus_signatures.append(signature)

end = time.time()

print(f"Runtime of the generating signatures was {end - start}")
print(f"Average per document: {(end - start) / len(documents_paths)}")

Runtime of the generating signatures was 68.9510555267334
Average per document: 3.44755277633667


In [61]:
# LSH
start = time.time()
candidate_pairs = lsh(corpus_signatures, signature_length, 0.9, max_hash_minhash)
end = time.time()

print(f"Runtime of LSH was {end - start}")
print(f"Average per document: {(end - start) / len(documents_paths)}")
candidate_pairs

Runtime of LSH was 0.05895352363586426
Average per document: 0.002947676181793213


{(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (11, 12), (14, 15), (16, 17)}

In [62]:
for candidate_pair in candidate_pairs:
  i = candidate_pair[0]
  j = candidate_pair[1]

  print("\t Candidate pair: " + str(candidate_pair))
  
  # Same method than on calculating signature
  jaccard_similarity = get_jaccard_similarity_from_shingles(corpus_shingles[i], corpus_shingles[j])

  print("\t Jaccard Similarity: " + str(jaccard_similarity))
  minhash_similarity = get_similarity_minhash_signatures(corpus_signatures[i], corpus_signatures[j])

  print("\t Minhash Signature Similarity: " + str(minhash_similarity))
  print("\n")

	 Candidate pair: (1, 2)
	 Jaccard Similarity: 0.8237303695454238
	 Minhash Signature Similarity: 0.947


	 Candidate pair: (16, 17)
	 Jaccard Similarity: 0.7132381915057651
	 Minhash Signature Similarity: 0.851


	 Candidate pair: (0, 1)
	 Jaccard Similarity: 0.8358193070928024
	 Minhash Signature Similarity: 0.947


	 Candidate pair: (1, 3)
	 Jaccard Similarity: 0.750593107609605
	 Minhash Signature Similarity: 0.863


	 Candidate pair: (2, 3)
	 Jaccard Similarity: 0.7935244377485511
	 Minhash Signature Similarity: 0.914


	 Candidate pair: (14, 15)
	 Jaccard Similarity: 0.7309323378549133
	 Minhash Signature Similarity: 0.855


	 Candidate pair: (11, 12)
	 Jaccard Similarity: 0.8013582241009721
	 Minhash Signature Similarity: 0.913


	 Candidate pair: (0, 2)
	 Jaccard Similarity: 0.7786234509056245
	 Minhash Signature Similarity: 0.894


